In [16]:
import cv2
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm_notebook as tqdm


In [42]:
data = []
with open('../darknet/predictions.txt','r') as f:
    for line in f.readlines():
        if line.startswith('Enter'):
            img_path = line.split('Path: ')[-1].split(': Predicted')[0].strip()
        if line.startswith('HWR'): 
            _type = 'HWR'
        elif line.startswith('QR'):
            _type = 'QR'
        elif line.startswith('Mix'):
            _type = 'Mix'
        elif line.startswith('OCR'):
            _type = 'OCR'
        if line.startswith('HWR') | line.startswith('QR') | line.startswith('Mix') | line.startswith('OCR'):
            if 'left_x' in line:
                _type = line.split(':')[0]
                certainty = int(line.split(':')[1].split('%')[0].strip())/100
                lx = int(line.split('left_x:')[1].split('top_y')[0].strip())
                ty = int(line.split('top_y:')[1].split('width')[0].strip())
                w = int(line.split('width:')[1].split('height')[0].strip())
                h = int(line.split('height:')[1].split(')')[0].strip())
                data += [[img_path,lx,ty,w,h,_type]]
        else:
            pass

In [36]:
row

lowres_fp    /home/rob/HWR/datasets/zooniverse/222064/Cropp...
lx                                                         949
ty                                                        1749
w                                                         2364
h                                                          860
type                                                        QR
dataset                                             zooniverse
Name: 9865, dtype: object

In [44]:
padding = 15
from glob import glob
from sklearn.cluster import KMeans
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
df = pd.DataFrame(data,columns=['lowres_fp','lx','ty','w','h','type'])#.to_csv('lowres_aarhus_bounding_boxes.csv')
print(len(df))
df['dataset'] = df['lowres_fp'].map(lambda x: 'Aarhus' if 'Aarhus' in x else 'zooniverse')
i = 0
for name,group in tqdm(df.groupby('lowres_fp'),total=len(df['lowres_fp'].drop_duplicates())):
    i = 0
    fp = name
    dataset = group['dataset'].iloc[0]

    img = cv2.imread(fp)
    for _, row in group.iterrows():
        sx = max(row['ty'] - padding,0)
        sy = max(row['lx'] - padding,0)
        ex = min(row['h'] + row['ty'] + padding, img.shape[0])
        ey = min(row['w'] + row['lx'] + padding, img.shape[1])
        _type = row['type']
        if dataset == 'Aarhus':
            cat_no = fp.split('-')[2]
            out_path = '/home/rob/HWR/datasets/Aarhus/yolo_labels/' + cat_no + '_' + str(i) + '_' + _type + '.jpg'
        elif dataset == 'zooniverse':
            cat_no = fp.split('/')[6]
            out_path = '/home/rob/HWR/datasets/zooniverse/yolo_labels/' + cat_no + '_' + str(i) + '_' + _type + '.jpg'

        label = cv2.cvtColor(img[sx:ex,sy:ey], cv2.COLOR_BGR2GRAY)

        kmeans = KMeans(n_clusters=2, n_init=1, init=np.array([[0],[255]])).fit(np.expand_dims(label.flatten(),axis=1))
        mask = kmeans.predict(label.reshape(-1,1)).reshape(label.shape)
        label[mask == 1] = 255
            
            
            
        cv2.imwrite(out_path, label)
        i+=1

9866


KeyboardInterrupt: 

In [29]:
df['cat_no'] = df['lowres_fp'].map(lambda x: x.split('-')[2] if 'Aarhus' in x else x.split('/')[6])
df[df['cat_no']=='222064']

,lowres_fp,lx,ty,w,h,type,dataset,cat_no
9864,/home/rob/HWR/datasets/zooniverse/222064/Cropp...,902,517,1699,1065,OCR,zooniverse,222064
9865,/home/rob/HWR/datasets/zooniverse/222064/Cropp...,949,1749,2364,860,QR,zooniverse,222064


In [30]:
df

,lowres_fp,lx,ty,w,h,type,dataset,cat_no
0,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199020-...,324,177,912,462,HWR,Aarhus,199020
1,/home/rob/HWR/datasets/zooniverse/222876/Cropp...,733,2391,1570,623,QR,zooniverse,222876
2,/home/rob/HWR/datasets/zooniverse/222876/Cropp...,904,1836,1067,603,HWR,zooniverse,222876
3,/home/rob/HWR/datasets/zooniverse/222638/Cropp...,464,1260,1873,833,OCR,zooniverse,222638
4,/home/rob/HWR/datasets/zooniverse/222638/Cropp...,973,2159,2196,821,QR,zooniverse,222638
5,/home/rob/HWR/datasets/zooniverse/222638/Cropp...,2143,1276,1184,757,OCR,zooniverse,222638
6,/home/rob/HWR/datasets/zooniverse/221584/Cropp...,855,2182,2062,784,QR,zooniverse,221584
7,/home/rob/HWR/datasets/zooniverse/221584/Cropp...,971,1156,1566,918,OCR,zooniverse,221584
8,/home/rob/HWR/datasets/zooniverse/221803/Cropp...,278,2155,2160,566,OCR,zooniverse,221803
9,/home/rob/HWR/datasets/zooniverse/221803/Cropp...,285,334,1731,717,OCR,zooniverse,221803
